In [ ]:
"""
Using Testing Data Instead Of Validation
"""
import os
import json
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, f1_score
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn
from lib.utils_simple import evaluate, compute_loss_and_f1
from lib.models import ConvLayerNorm, Block
import matplotlib.pyplot as plt

class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        self.blocks = []
        self.blocks.append(Block(6,8))
        for _ in range(5):
            self.blocks.append(Block(8,8))
            self.blocks.append(Block(8,8,pool=False))

        self.blocks.append(Block(8,16,pool=False))

        # for _ in range(5):
        #     self.blocks.append(Block(16,16))
        #     self.blocks.append(Block(16,16,pool=False))
            
        self.blocks = nn.ModuleList(self.blocks)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(16, 1)
    
    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return x
    
# Load all experiments into a dataframe
performances = {}
from tqdm import tqdm

experiments_dir = f'./experiments_all_participants'

for experiment in tqdm(os.listdir(experiments_dir)):
    base_f1s = []
    target_f1s = []
    folds = []

    for run in os.listdir(f'{experiments_dir}/{experiment}'):
        if not os.path.exists(f'{experiments_dir}/{experiment}/{run}/metrics.json'):
            continue
        metrics = json.load(open(f'{experiments_dir}/{experiment}/{run}/metrics.json'))
        losses = json.load(open(f'{experiments_dir}/{experiment}/{run}/losses.json'))
        hyperparameters = json.load(open(f'{experiments_dir}/{experiment}/{run}/hyperparameters.json'))

        data_path = hyperparameters['data_path']
        target_participant = hyperparameters['target_participant']
        target_testloader = DataLoader(TensorDataset(*torch.load(f'data/001_60s_window/{target_participant}_test.pt')), batch_size=128, shuffle=False)

        model = TestModel()
        criterion = nn.BCEWithLogitsLoss()

        if 'target_only' in experiment:
            base_model_on_target_val_f1 = None
            base_f1s.append(base_model_on_target_val_f1)
            model.load_state_dict(torch.load(f'{experiments_dir}/{experiment}/{run}/best_base_model.pt', map_location='cpu')) # naming is an artifact of training mode
            _,target_model_on_target_val_f1 = compute_loss_and_f1(model, target_testloader, criterion, device='cpu')
            target_f1s.append(target_model_on_target_val_f1)

        else:
            model.load_state_dict(torch.load(f'{experiments_dir}/{experiment}/{run}/best_base_model.pt', map_location='cpu'))
            _,base_model_on_target_val_f1 = compute_loss_and_f1(model, target_testloader, criterion, device='cpu')
            base_f1s.append(base_model_on_target_val_f1)

            model.load_state_dict(torch.load(f'{experiments_dir}/{experiment}/{run}/best_target_model.pt', map_location='cpu'))
            _,target_model_on_target_val_f1 = compute_loss_and_f1(model, target_testloader, criterion, device='cpu')

            target_f1s.append(target_model_on_target_val_f1)

        folds.append(run)

    performances[experiment] = (base_f1s, target_f1s, folds)

data = []
for experiment, (base_f1s, target_f1s, folds) in performances.items():
    for base_f1, target_f1, fold in zip(base_f1s, target_f1s, folds):
        data.append({'experiment': experiment, 'model': 'base', 'f1': base_f1, 'fold': fold})
        data.append({'experiment': experiment, 'model': 'target', 'f1': target_f1, 'fold': fold})

df_original_testing = pd.DataFrame(data)

In [ ]:
df = df_original_testing.copy()
df.dropna(inplace=True)
df = df[~(df['model'] == 'base')]
df.loc[df['experiment'].str.contains('full'),'model'] = 'full fine tuning'
df.loc[df['experiment'].str.contains('target'),'model'] = 'target only'
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)
df

sns.lineplot(x='target training data percentage', y='f1', hue='model', marker='o', data=df)

In [ ]:
df = df_original_testing.copy()
df = df[df['experiment'].str.contains('full_fine_tuning')]
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)
df

In [ ]:
sns.barplot(data=df, x='fold', y='f1', hue='model')

In [ ]:
sns.boxplot(data=df, x='target training data percentage', y='f1', hue='model')

In [ ]:
import matplotlib.pyplot as plt
df = df_original_testing.copy()
df = df[df['experiment'].str.contains('full_fine_tuning')]
df.dropna(inplace=True)
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)

sns.boxplot(data=df, x='target training data percentage', y='f1', hue='model')
plt.title('')

In [ ]:
import matplotlib.pyplot as plt
df = df_original_testing.copy()
df = df[df['experiment'].str.contains('full_fine_tuning')]
df.dropna(inplace=True)
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)

sns.boxplot(data=df, x='fold', y='f1', hue='model')
plt.title('F1 Score vs Fold')

In [ ]:
df = df_original_testing.copy()
df = df[df['experiment'].str.contains('full_fine_tuning_pct0.05')]
df.dropna(inplace=True)
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)
df
# sns.lineplot(data=df, x='target training data percentage', y='f1', hue='fold')
# plt.title('F1 Score vs Target Training Data Percentage')

In [ ]:

# df = df[df['fold'].str.contains('fold2')]
df = df[~(df['model'] == 'base')]
df.loc[df['experiment'].str.contains('full'),'model'] = 'full fine tuning'
df.loc[df['experiment'].str.contains('target'),'model'] = 'target only'

df

sns.lineplot(x='target training data percentage', y='f1', hue='model', marker='o', data=df)

In [ ]:

df = df_original_testing.copy()
df.dropna(inplace=True)
df = df[df['fold'].str.contains('fold2')]
df = df[~(df['model'] == 'base')]
df.loc[df['experiment'].str.contains('full'),'model'] = 'full fine tuning'
df.loc[df['experiment'].str.contains('target'),'model'] = 'target only'
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)
df

sns.lineplot(x='target training data percentage', y='f1', hue='model', marker='o', data=df)

In [ ]:
"""
Using Testing Data Instead Of Validation
"""
import os
import json
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, f1_score
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn
from lib.utils_simple import evaluate, compute_loss_and_f1
from lib.models import ConvLayerNorm, Block
import matplotlib.pyplot as plt

class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        self.blocks = []
        self.blocks.append(Block(6,8))
        for _ in range(5):
            self.blocks.append(Block(8,8))
            self.blocks.append(Block(8,8,pool=False))

        self.blocks.append(Block(8,16,pool=False))

        # for _ in range(5):
        #     self.blocks.append(Block(16,16))
        #     self.blocks.append(Block(16,16,pool=False))
            
        self.blocks = nn.ModuleList(self.blocks)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(16, 1)
    
    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return x
    
# Load all experiments into a dataframe
performances = {}
from tqdm import tqdm

experiments_dir = f'./experiments_3_participants'

for experiment in tqdm(os.listdir(experiments_dir)):
    base_f1s = []
    target_f1s = []
    folds = []

    for run in os.listdir(f'{experiments_dir}/{experiment}'):
        if not os.path.exists(f'{experiments_dir}/{experiment}/{run}/metrics.json'):
            continue
        metrics = json.load(open(f'{experiments_dir}/{experiment}/{run}/metrics.json'))
        losses = json.load(open(f'{experiments_dir}/{experiment}/{run}/losses.json'))
        hyperparameters = json.load(open(f'{experiments_dir}/{experiment}/{run}/hyperparameters.json'))

        data_path = hyperparameters['data_path']
        target_participant = hyperparameters['target_participant']
        target_testloader = DataLoader(TensorDataset(*torch.load(f'{data_path}/{target_participant}_test.pt')), batch_size=128, shuffle=False)

        model = TestModel()
        criterion = nn.BCEWithLogitsLoss()

        if 'target_only' in experiment:
            base_model_on_target_val_f1 = None
            base_f1s.append(base_model_on_target_val_f1)
            model.load_state_dict(torch.load(f'{experiments_dir}/{experiment}/{run}/best_base_model.pt', map_location='cpu')) # naming is an artifact of training mode
            _,target_model_on_target_val_f1 = compute_loss_and_f1(model, target_testloader, criterion, device='cpu')
            target_f1s.append(target_model_on_target_val_f1)

        else:
            model.load_state_dict(torch.load(f'{experiments_dir}/{experiment}/{run}/best_base_model.pt', map_location='cpu'))
            _,base_model_on_target_val_f1 = compute_loss_and_f1(model, target_testloader, criterion, device='cpu')
            base_f1s.append(base_model_on_target_val_f1)

            model.load_state_dict(torch.load(f'{experiments_dir}/{experiment}/{run}/best_target_model.pt', map_location='cpu'))
            _,target_model_on_target_val_f1 = compute_loss_and_f1(model, target_testloader, criterion, device='cpu')

            target_f1s.append(target_model_on_target_val_f1)

        folds.append(run)

    performances[experiment] = (base_f1s, target_f1s, folds)

data = []
for experiment, (base_f1s, target_f1s, folds) in performances.items():
    for base_f1, target_f1, fold in zip(base_f1s, target_f1s, folds):
        data.append({'experiment': experiment, 'model': 'base', 'f1': base_f1, 'fold': fold})
        data.append({'experiment': experiment, 'model': 'target', 'f1': target_f1, 'fold': fold})

df_original = pd.DataFrame(data)


In [ ]:
import os
import json

performances = {}
for experiment in os.listdir('./experiments'):
    base_f1s = []
    target_f1s = []
    folds = []
    for run in os.listdir(f'./experiments/{experiment}'):
        if not os.path.exists(f'./experiments/{experiment}/{run}/metrics.json'):
            continue
        metrics = json.load(open(f'./experiments/{experiment}/{run}/metrics.json'))
        losses = json.load(open(f'./experiments/{experiment}/{run}/losses.json'))

        if 'target_only' in experiment:
            target_model_on_target_val_f1 = losses['target val f1'][metrics['best_base_val_loss_epoch']]
            base_model_on_target_val_f1 = None
        else:
            target_model_on_target_val_f1 = losses['target val f1'][metrics['best_target_val_loss_epoch']]
            base_model_on_target_val_f1 = losses['target val f1'][metrics['best_base_val_loss_epoch']]

        base_f1s.append(base_model_on_target_val_f1)
        target_f1s.append(target_model_on_target_val_f1)
        folds.append(run)
    performances[experiment] = (base_f1s, target_f1s, folds)

# boxplot all performances
import pandas as pd
data = []
for experiment, (base_f1s, target_f1s, folds) in performances.items():
    for base_f1, target_f1, fold in zip(base_f1s, target_f1s, folds):
        data.append({'experiment': experiment, 'model': 'base', 'f1': base_f1, 'fold': fold})
        data.append({'experiment': experiment, 'model': 'target', 'f1': target_f1, 'fold': fold})
df_original = pd.DataFrame(data)

import seaborn as sns

In [ ]:
df_original

In [ ]:

df = df_original.copy()
df.dropna(inplace=True)
# df = df[df['fold'].str.contains('fold2')]
df = df[~(df['model'] == 'base')]
df.loc[df['experiment'].str.contains('full'),'model'] = 'full fine tuning'
df.loc[df['experiment'].str.contains('target'),'model'] = 'target only'
df['experiment'] = df['experiment'].str.split('_').str[-3].str.replace('pct','').astype(float)
df.rename(columns={'experiment':'target training data percentage'}, inplace=True)
df

sns.lineplot(x='target training data percentage', y='f1', hue='model', marker='o', data=df)

In [ ]:
import os
import json

"""
for each full fine tuning experiment,
"""
performances = {}
for experiment in os.listdir('./experiments'):
    if 'full_fine_tuning' not in experiment:
        continue
    
    base_f1s = []
    target_f1s = []
    folds = []
    for run in os.listdir(f'./experiments/{experiment}'):
        if not os.path.exists(f'./experiments/{experiment}/{run}/metrics.json'):
            continue
        metrics = json.load(open(f'./experiments/{experiment}/{run}/metrics.json'))
        losses = json.load(open(f'./experiments/{experiment}/{run}/losses.json'))

        if 'target_only' in experiment:
            target_model_on_target_val_f1 = losses['target val f1'][metrics['best_base_val_loss_epoch']]
            base_model_on_target_val_f1 = None
        else:
            target_model_on_target_val_f1 = losses['target val f1'][metrics['best_target_val_loss_epoch']]
            base_model_on_target_val_f1 = losses['target val f1'][metrics['best_base_val_loss_epoch']]

        base_f1s.append(base_model_on_target_val_f1)
        target_f1s.append(target_model_on_target_val_f1)
        folds.append(run)
    performances[experiment] = (base_f1s, target_f1s, folds)

# boxplot all performances
import pandas as pd
data = []
for experiment, (base_f1s, target_f1s, folds) in performances.items():
    for base_f1, target_f1, fold in zip(base_f1s, target_f1s, folds):
        data.append({'experiment': experiment, 'model': 'base', 'f1': base_f1, 'fold': fold})
        data.append({'experiment': experiment, 'model': 'target', 'f1': target_f1, 'fold': fold})
df_original = pd.DataFrame(data)

import seaborn as sns